In [1]:
import spacy
import pandas as pd
import string
import pickle
import textacy

C:\Users\kondr\anaconda3\envs\tf2.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kondr\anaconda3\envs\tf2.8\lib\site-packages\torch\_masked\__init__.py:223: UserWarning: Failed to initialize NumPy: module compiled against API version 0xf but this version of numpy is 0xe (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  example_input = torch.tensor([[-3, -2, -1], [0, 1, 2]])


In [2]:
# from sentence_transformers import SentenceTransformer

In [3]:
from numba import jit,njit, vectorize,cuda,uint32, f8, uint8

In [4]:
import timeit

In [5]:
data = pd.read_csv("C:/Users/kondr/Desktop/Ramya/_USML_S22/project/L1haveimg.csv")

In [6]:
len(data)

402760

In [7]:
data.head()

,ingredients,url,partition,title,id,instructions
0,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...
1,"[{'text': '2 Chicken thighs'}, {'text': '2 tsp...",https://cookpad.com/us/recipes/150100-kombu-te...,train,Kombu Tea Grilled Chicken Thigh,000075604a,[{'text': 'Pierce the skin of the chicken with...
2,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",http://www.food.com/recipe/strawberry-rhubarb-...,train,Strawberry Rhubarb Dump Cake,00007bfd16,[{'text': 'Put ingredients in a buttered 9 x 1...
3,"[{'text': '8 ounces, weight Light Fat Free Van...",http://tastykitchen.com/recipes/breakfastbrunc...,train,Yogurt Parfaits,000095fc1d,[{'text': 'Layer all ingredients in a serving ...
4,"[{'text': '1 teaspoon fennel seeds'}, {'text':...",http://www.epicurious.com/recipes/food/views/f...,train,Fennel-Rubbed Pork Tenderloin with Roasted Fen...,0000b1e2b5,[{'text': 'Preheat oven to 350F with rack in m...


In [81]:
@jit
def check1(n):
    return n+n
        

In [82]:
def check2(n):
    return n*n

In [83]:
start=timeit.default_timer()
check1(1000)
end=timeit.default_timer()-start
print(end)


0.03872179999598302


In [84]:
start=timeit.default_timer()
check2(1000)
end=timeit.default_timer()-start
print(end)


5.23999915458262e-05


In [ ]:
import numpy as np

In [56]:
x = np.array([1, 2, 2.5], dtype=np.float32)
type(x)
# x.astype(float)

numpy.ndarray

In [74]:
start=timeit.default_timer()
threadsperblock = 32
blockspergrid = (an_array.size + (threadsperblock - 1)) // threadsperblock
check2[blockspergrid, threadsperblock](1000)

end=timeit.default_timer()-start
print(end)

C:\Users\kondr\anaconda3\envs\tf2.8\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (28) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
[1m[1mNo conversion from int64 to none for '$8return_value.3', defined at None
[1m
File "..\..\AppData\Local\Temp\ipykernel_13844\3201734810.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of assignment at C:\Users\kondr\AppData\Local\Temp\ipykernel_13844\3201734810.py (3)[0m
[1m
File "..\..\AppData\Local\Temp\ipykernel_13844\3201734810.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m


In [147]:
#loading spacy library
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")


In [18]:

#loading csv
data = pd.read_csv("C:/Users/kondr/Desktop/Ramya/_USML_S22/project/L1haveimg.csv")
  
def cleaning_ing(ing):
    cleanings=[]
    # print(ing)
    
    for clean in ing:
        cleanitems=[]
        
        #tokenizing ingredients
        mytokens=clean.split("}, ")
        newtokens=[]
        
        for i in mytokens:
            i=i.lower()
            j=nlp(i)
            newtokens.append([k.text for k in j if k.is_stop==False])
        
        # print(newtokens)

        for i in newtokens:
            # print(f"i:{i}")
            ingredients=[]

            #cleaning ingredients
            remove_punc=["text","}","[","]", "'", "(",")","-","/", ",",".",":",";","{","|","\\","=","+","_","<",">","!","@","#","$","%","^","&","*","~","`"]
            remove_measures=["fluids","fluid","fluid ounces","grams","gram","cartons","carton","sticks","stick","liters","liter","inches","inch","cups","cup","ounces","ounce","teaspoons","teaspoon","tablespoons","tablespoon","pints","pint","quartered","quarter","quarts","quart","gallons","gallon","pounds","pound","pinch","dash","boxes","box","cans","can"]
            remove_shrt_msr=["tsp","tbs","tbsp","tsps","tbsps","lb","lbs","oz","cc","ml","fl. oz","fl","fl.oz"]
            
            # for tok in i:
            i1 = list(filter(lambda tok: tok not in remove_punc  , i))
            i2 = list(filter(lambda tok: tok not in remove_measures  , i1))
            j = list(filter(lambda tok: tok not in remove_shrt_msr  , i2))
            
            for tok in j:
                # print(tok)
                if tok!="" and tok.isdigit()==False:
                    # print("no digits")
                    if set(tok).intersection(set(string.punctuation)) == set():
                        # print(tok)
                        ingredients.append(tok)
                    # ingredients.append(tok)
            
            
            
            cleanitems.append(" ".join(ingredients))
        cleanings.append(cleanitems)
    return(cleanings)



In [7]:
len(data["ingredients"])

402760

In [14]:
%%time
#open text file
with open('C:/Users/kondr/Desktop/Ramya/_USML_S22/project/test3_TF.txt', 'wb') as f:
    clean_ingredients = cleaning_ing(data["ingredients"][402755:])
    pickle.dump(clean_ingredients, f)

f.close()

# print(clean_ingredients)

CPU times: total: 328 ms
Wall time: 332 ms


In [8]:
%%time
with open('C:/Users/kondr/Desktop/Ramya/_USML_S22/project/cleaning1_ing_TF.txt', 'rb') as f:
    dish = pickle.load(f)
f.close()

# print(dish)

CPU times: total: 1.22 s
Wall time: 1.23 s


In [85]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
type(dish)

list

In [13]:
len(dish)

402760

In [14]:
dish[0]

['milk',
 'water',
 'butter',
 'mashed potatoes homestyle',
 'kernel corn drained',
 'cheddar cheese',
 'french fried onions']

In [128]:
test1000 = dish[0:1000]
# print(test1000)

In [114]:
# pre_pattern = [{"POS":{'IN': ['NOUN','PROPN']}}]

# for ings in test1000:
#     for n in ings:
#         check = nlp(n)
#         print("--------------------------")
#         for i in check:
#             print(f'{i}:\t{i.pos_}')
#         test1 = textacy.extract.matches.token_matches(check,pre_pattern)
#         for i in test1:
#             print(i)

In [150]:
#extracting nouns and proper nouns
def extract_nouns(data):
    mypatterns =  [{"POS":{'IN': ['NOUN','PROPN']}}]
    
    clean_nouns = []
    clean_nouns3max=[]
    
    for ings in data:
        items=[]
        items_3max=[]
        for i in ings:
            n=[]
            check= nlp(i)
            nouns = textacy.extract.matches.token_matches(check, patterns=mypatterns)

            for i in nouns:
                n.append(str(i))
            if len(n)==0:
                pass
            if len(n)==1:
                if (n[0] not in items)==True:
                    items.append(n[0])
                    items_3max.append(n[0])
            else:
                x=" ".join(n)
                if (x not in items)==True:
                    items.append(x)
                if (len(n)<4):
                    if (x not in items_3max)==True:
                        items_3max.append(x)
        clean_nouns.append(items)
        clean_nouns3max.append(items_3max)
    return clean_nouns, clean_nouns3max

In [151]:

start=timeit.default_timer()
a,b=extract_nouns(dish[0:100])




Time taken:	3.1895584000012605


In [132]:
print(a)
print(b)

[['milk', 'water', 'butter', 'potatoes homestyle', 'kernel corn', 'cheddar cheese', 'french onions'], ['chicken thighs', 'kombu tea', 'white pepper'], ['rhubarb', 'sugar', 'strawberry jell o gelatin dessert', 'white cake mix layer size', 'water', 'butter margarine'], ['weight vanilla yogurt activia', 'strawberries', 'granola'], ['fennel seeds', 'pork tenderloin', 'fennel bulbs reserving fronds', 'olive oil', 'garlic cloves', 'wine', 'sodium chicken broth', 'butter cut pieces', 'lemon juice taste']]
[['milk', 'water', 'butter', 'potatoes homestyle', 'kernel corn', 'cheddar cheese', 'french onions'], ['chicken thighs', 'kombu tea', 'white pepper'], ['rhubarb', 'sugar', 'water', 'butter margarine'], ['strawberries', 'granola'], ['fennel seeds', 'pork tenderloin', 'olive oil', 'garlic cloves', 'wine', 'sodium chicken broth', 'butter cut pieces', 'lemon juice taste']]


In [153]:
%%time
start=timeit.default_timer()
clean_nouns,clean_nouns3max = extract_nouns(dish)
total=timeit.default_timer()-start
print(f"Time taken:\t{total}")

Time taken:	14089.727289300004
CPU times: total: 3h 54min 49s
Wall time: 3h 54min 49s


In [154]:
%%time
#open text file
with open('C:/Users/kondr/Desktop/Ramya/_USML_S22/project/extractNOUNS_TF.txt', 'wb') as f:
    pickle.dump(clean_nouns, f)

#open text file
with open('C:/Users/kondr/Desktop/Ramya/_USML_S22/project/extractNOUNS3max_TF.txt', 'wb') as f:
    pickle.dump(clean_nouns3max, f)



CPU times: total: 2.67 s
Wall time: 2.68 s


In [87]:
with open('C:/Users/kondr/Desktop/Ramya/_USML_S22/project/extractNOUNS3max_TF.txt', 'rb') as f:
    data2 = pickle.load(f)


In [37]:
len(data2)

402760

In [98]:
data3 = pd.DataFrame(columns=["title","clean_ingr"])

In [99]:
data3

,title,clean_ingr


In [ ]:
start=timeit.default_timer()
for ind, i in enumerate(data2):
    data3.at[ind,"clean_ingr"]=i
    data3.at[ind,"title"]=data["title"][ind]
end=timeit.default_timer()-start
print(end)

In [ ]:
data3